<a href="https://colab.research.google.com/github/azfar154/Standard-Vector-Machine-Tensorflow/blob/master/SVC_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
class SVC():
  def __init__(self,C= 5):
    self.C =5

  def fit(self,X_train,y_train,show_weights = False,show_loss = False,epochs = 300000):    
    batch_size = 300
    features = X_train.shape[1]
    print("The amount of features are",features)
    self.features= features
    x_data = tf.placeholder(shape=[None, features], dtype=tf.float32)
    self.x_data = x_data
    y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
    self.y_target = y_target
    W = tf.Variable(tf.random_normal(shape=[features, 1]))
    b = tf.Variable(tf.random_normal(shape=[1, 1]))
    model_output = tf.subtract(tf.matmul(x_data, W), b)

    l2_norm = tf.reduce_sum(tf.square(W))

    alpha_param = tf.constant([0.01])
    classification_term = tf.reduce_mean(tf.maximum(0., tf.subtract(1., tf.multiply(model_output, y_target))))
    loss = tf.add(classification_term,self.C * tf.multiply(alpha_param, l2_norm))

    prediction = tf.sign(model_output)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction,y_train),tf.float32))
    self.accuracy = accuracy
    residuals = prediction - y_train
    with tf.name_scope("Loss"):
      tf.summary.histogram("Histogram_Errors",accuracy)
      tf.summary.histogram("Histagram_Residuals",residuals)
      loss_summary_op = tf.summary.scalar("loss",loss[0])
    summary_op = tf.summary.merge_all()
    my_opt = tf.train.GradientDescentOptimizer(0.01)
    train_step = my_opt.minimize(loss)
    X_train = X_train.to_numpy()
    y_train = y_train.to_numpy()
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    display_step = 500
    # Training loop
    loss_vec = []
    train_accuracy = []
    test_accuracy = []
    epochs= epochs
    for i in range(epochs):
        rand_index = np.random.choice(len(X_train), size=batch_size)
        rand_x = X_train[rand_index]
        rand_y = np.transpose([y_train[rand_index]])
        sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
        if i % display_step == 0:
          weights = sess.run(W)
          if show_weights == True:
            for i in range(len(weights)):
              for x in weights[i]:
                print("Weight ",i,x)
          #test_loss, test_residuals = sess.run([loss,residuals],feed_dict={x_data:x_test,y_target:y_test})
          if show_loss == True:
            print( "B:",sess.run(b)[0][0])
            loss_percentage = sess.run(loss,feed_dict={x_data:rand_x,y_target:rand_y})
            print("Loss :",loss_percentage[0])
    self.sess = sess
    self.weights = sess.run(W)
    self.bias = sess.run(b)
    self.loss = sess.run(loss,feed_dict={x_data:X_train,y_target:np.transpose([y_train])})
    print("Training Accuracy =",self.loss[0])
  def show_stats_train(self):
    return self.sess,self.weights,self.bias,self.loss
  def score(self,x_test,y_test):
    x_test = x_test.to_numpy()
    y_test = y_test.to_numpy()
    test_acc = self.sess.run(self.accuracy,feed_dict={self.x_data:x_test,self.y_target:np.transpose([y_test])})
    return test_acc


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('heart.csv')

In [4]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [0]:
target = data['target']

In [6]:
target.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

In [75]:
X_train

array([[-0.30209462,  2.01531366, -1.58427484, ...,  0.29079054,
        -1.19856641, -0.39366662],
       [-2.1282656 ,  0.09506976, -0.5984443 , ...,  0.22980857,
         0.25025563,  0.52288809],
       [ 0.07934376, -0.4594317 ,  0.04933809, ..., -0.82251854,
        -0.88919194,  0.23465154],
       ...,
       [ 1.05082659,  0.13350826, -1.63727427, ...,  0.57847759,
        -0.68486815,  0.30741838],
       [ 1.24798795, -2.26983814,  1.0076397 , ...,  0.69855388,
         1.28543998,  1.25827079],
       [-0.206818  , -1.63102082, -0.09973378, ..., -0.48744382,
        -1.56557381, -0.09752086]])

In [0]:
features = data.drop(columns=['target'])

In [0]:
from sklearn.model_selection import train_test_split
X_train,x_test,y_train,y_test = train_test_split(features,target,random_state = 0 )

In [0]:
model = SVC()

In [72]:
model.fit(X_train,y_train)

The amount of features are 13
Training Accuracy = 0.46255696


In [61]:
model.score(x_test,y_test)

0.53744495

In [0]:
from sklearn.svm import SVC

In [63]:
model = SVC().fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [64]:
model.score(x_test,y_test)

0.5921052631578947

In [0]:
from sklearn.dummy import DummyClassifier

In [0]:
dummy_clf = DummyClassifier(strategy="most_frequent").fit(X_train,y_train)

In [57]:
dummy_clf.score(x_test,y_test)

0.5657894736842105